In [7]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

# from pathlib import Path
import h5py

Using TensorFlow backend.


加载模型权重

In [11]:
model_weight_resnet50 = "model_ResNet50.h5"
model_weight_xception = "model_Xception.h5"
model_weight_inceptionV3 = "model_InceptionV3.h5"

建立辅助函数，提取特征向量

In [30]:
def write_gap(MODEL,model_weight, image_size, preprocessing=None, filename_option=""):
    # parameters
    batch_size = 64
    # path_dataset = Path('dataset')
    path_train = "dataset/train2"
    path_test = "dataset/valid2"
    
    # set up Model
    print('set up Model')
    height = image_size[0]
    width = image_size[1]
    
    base_model = MODEL(input_tensor=Input((height, width, 3)),
                            weights=None, 
                            include_top=False, 
                            input_shape=(height, width, 3)
                           )
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    
    # load weight
    print('load weight')
    model.load_weights("models/" + model_weight, by_name=True)
    
    train_gen = ImageDataGenerator(
        rotation_range=10.,
        width_shift_range=0.05,
        height_shift_range=0.05,
        shear_range=0.05,
        zoom_range=0.05,
        preprocessing_function=preprocessing
    )
    
    test_gen = ImageDataGenerator(
        preprocessing_function=preprocessing
    )
    
    print('read dataset')
    train_generator = train_gen.flow_from_directory(
        path_train, (height, width), shuffle=False, batch_size=batch_size)
    test_generator = test_gen.flow_from_directory(
        path_test, (height, width), shuffle=False, batch_size=batch_size)
    
    # predictions
    print('make predictions')
    steps_train_sample = train_generator.samples // batch_size + 1
    steps_test_sample = test_generator.samples // batch_size + 1
    train = model.predict_generator(train_generator, steps=steps_train_sample)
    test = model.predict_generator(test_generator, steps=steps_test_sample)
    
    # save features
    
    print('save features')
    path_finetune_feature = 'features/finetune_{}{}.h5'.format(MODEL.__name__, filename_option)
    with h5py.File(path_finetune_feature) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("valid", data=test)
        h.create_dataset("label", data=train_generator.classes)
        h.create_dataset("valid_label", data=test_generator.classes)
    print("write_gap {} finsh".format(Model.__name__))

In [32]:
write_gap(ResNet50, model_weight_resnet50, (224, 224))

set up Model
load weight
read dataset
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
make predictions
save features
write_gap Model finsh


TypeError: 'NoneType' object is not iterable

In [33]:
write_gap(Xception, model_weight_xception, (299, 299), preprocessing=xception.preprocess_input)

set up Model
load weight
read dataset
Found 20787 images belonging to 10 classes.
Found 1637 images belonging to 10 classes.
make predictions
save features


IOError: Can't prepare for writing data (File write failed: time = sat feb  3 17:47:04 2018
, filename = 'features/finetune_xception.h5', file descriptor = 24, errno = 28, error message = 'no space left on device', buf = 0x7f4867e737b0, total write size = 114866272, bytes this sub-write = 114866272, bytes actually written = 18446744073709551615, offset = 55422976)

In [ ]:
write_gap(InceptionV3, model_weight_inceptionV3, (299, 299), preprocessing=inception_v3.preprocess_input)